In [39]:
import requests
import pandas as pd
from config import tmdb_key
from pprint import pprint as pprint

In [40]:
base_params = dict(api_key=tmdb_key)
year = 2019
page = 1
listing_params = dict(primary_release_year=year,region='US', sort_by='revenue.desc', include_adult='false', language='en-US', page='1')
listing_params.update(base_params)
response = requests.get('https://api.themoviedb.org/3/discover/movie', params=listing_params)
imdb_films = response.json()['results']


In [41]:
rows = []

restart = True
page_limit = True
while restart == True:
   while page_limit == True:
      for film in imdb_films:
         film_revenue = requests.get('https://api.themoviedb.org/3/movie/{}'.format(film['id']), params=listing_params)
         film_revenue = film_revenue.json()
         try:
            row = dict(title=film['title'], revenue=film_revenue['revenue'], Year=listing_params['primary_release_year'],\
                     VoteAVG=film['vote_average'], VoteCT=film['vote_count'], Popularity=film['popularity'], Date=film['release_date'],\
                        Genre=film['genre_ids'][0])
         except:
            pass   
         rows.append(row)
      #MODIFY page == x: to increase or decrease sample size
      if page == 10: 
         page_limit = False
      else:
         page += 1
         listing_params.update({'page':page})
         response = requests.get('https://api.themoviedb.org/3/discover/movie', params=listing_params)
         imdb_films = response.json()['results']
   if year == 2023: 
      restart = False
   else:
      page_limit = True
      page = 1
      year += 1
      listing_params.update({'primary_release_year':year})
      listing_params.update({'page':page})
      response = requests.get('https://api.themoviedb.org/3/discover/movie', params=listing_params)
      imdb_films = response.json()['results']
      
df = pd.DataFrame(rows, columns=['title', 'revenue', 'Year','VoteAVG', 'VoteCT', 'Popularity', 'Date', 'Genre'])
df['Date'] = pd.to_datetime(df['Date'])

In [42]:
#Copy df
df_copy = df.copy()

In [43]:
data = pd.DataFrame(
       {
              'A': [28,12,16,35,80,99,18,10751,14,36,27,10402,9648,10749,878,10770,53,10752,37],
              'B':['Action','Adventure','Animation','Comedy','Crime','Documentary','Drama','Family','Fantasy','History',\
                     'Horror','Music','Mystery','Romance','Science', 'Fiction','TV Movie','War','Western']
       }
)
map_dict = {28:'Action', 12:'Adventure'}
map_dict = {x:data['B'].tolist()[i] for i,x in enumerate(data['A'].tolist())}
df_copy['Genre'] = df_copy['Genre'].map(map_dict)
df_copy.head()

,title,revenue,Year,VoteAVG,VoteCT,Popularity,Date,Genre
0,Avengers: Endgame,2800000000,2019,8.3,24055,146.438,2019-04-26,Adventure
1,The Lion King,1663075401,2019,7.1,9402,80.841,2019-07-19,Adventure
2,Frozen II,1450000000,2019,7.3,9121,98.549,2019-11-22,Family
3,Spider-Man: Far From Home,1131927996,2019,7.4,14617,85.298,2019-07-02,Action
4,Captain Marvel,1131416446,2019,6.8,14775,105.741,2019-03-08,Action


In [44]:
df_copy.to_csv("output_data/tmdb_movies.csv", index_label="Index")

In [45]:
df_grouped = df_copy.groupby('Year')['revenue']
Mean = df_grouped.mean()
Median = df_grouped.median()
SUM = df_grouped.sum()
VAR = df_grouped.var()
STD = df_grouped.std()
SEM = df_grouped.sem()

summary_dic = {
    'Sum': SUM,
    'Mean': Mean,
    'Median': Median,
    'VAR': VAR,
    "STD": STD,
    'SEM': SEM
}

df_summary = pd.DataFrame(summary_dic)
df_summary

,Sum,Mean,Median,VAR,STD,SEM
Year,,,,,,
2019,29961648503,1.498082e+08,38170505.5,1.039134e+17,3.223560e+08,2.279401e+07
2020,6018720886,3.009360e+07,2088769.5,6.557609e+15,8.097906e+07,5.726084e+06
2021,14685477810,7.342739e+07,13414265.0,3.454449e+16,1.858615e+08,1.314239e+07
2022,19583301146,9.791651e+07,14789929.0,6.314694e+16,2.512905e+08,1.776892e+07
2023,18944537816,9.472269e+07,10950000.0,4.351155e+16,2.085942e+08,1.474984e+07
